No diretório raiz é possivel encontrar dois arquivos .md que explicam, de forma resumida, alguns pontos principais do projeto. 

De maneira geral, a simulação do processador se da através da execução de alguns código escritos em C, neste caso, existem alguns exemplos de cógidos prontos e adaptados para o atual sistema presentes no diretório SIM/FIRMWARE:

In [7]:
!cd SIM/FIRMWARE/ && ls

COREMARK      errno.c	perf.S	     putchar.S	       wait.S
coremark.c    Makefile	pipeline.ld  raystones.c
DHRYSTONE     memcpy.c	PRECOMPILED  start_pipeline.S
dhrystones.c  perf.h	print.c      test_rdcycle.c


Porém, antes da simulação é necessária a compilação desses para a arquitetura correta, respeitando os padrões de memória e runtime estipulados durante a criação do processador. No arquivo Makefile, presente no diretório SIM/FIRMWARE, encontra-se algumas regras que são utilizadas no processo de compilação.

In [8]:
!cd SIM/FIRMWARE/ && cat Makefile

include ../../FIRMWARE/makefile.inc
RVASFLAGS=-march=$(ARCH) -mabi=$(ABI) 
RVCFLAGS=-I. -O2 -fno-pic -march=$(ARCH) -mabi=$(ABI) -fno-stack-protector -w -Wl,--no-relax

RAM_SIZE=6144

LIBOBJECTS=putchar.o wait.o print.o memcpy.o errno.o perf.o

# DUAL MEMORY (64 kb program ROM, 64 kb data RAM)
%.pipeline.elf: %.o start_pipeline.o $(LIBOBJECTS) $(RV_BINARIES)
	$(RVLD) -T pipeline.ld -m elf32lriscv -nostdlib -norelax $< $(LIBOBJECTS) -L$(RVTOOLCHAIN_LIB_DIR) -lm $(RVTOOLCHAIN_GCC_LIB_DIR)/libgcc.a  -o $@
	$(RVOBJDUMP) -Mnumeric -D $@ > $@.list

%.PROGROM.hex: %.pipeline.elf $(FIRMWARE_DIR)/TOOLS/firmware_words 
	$(FIRMWARE_DIR)/TOOLS/firmware_words $< -ram 0x20000 -max_addr 0x20000 -out $@ -from_addr 0 -to_addr 0xFFFF
	cp $@ ../PROGROM.hex
	mkdir -p ../obj_dir
	cp $@ ../obj_dir/PROGROM.hex

%.DATARAM.hex: %.pipeline.elf $(FIRMWARE_DIR)/TOOLS/firmware_words 
	$(FIRMWARE_DIR)/TOOLS/firmware_words $< -ram 0x20000 -max_addr 0x20000 -out $@ -from_addr 0x10000 -to_addr 0x1FFFF
	cp $@ ../DATARA

Para compilar qualquer dos códigos é preciso utilizar a regra (programa).pipeline.hex, que, de acordo com o Makefile, tem como dependências a criação de dois arquivos (PROGROM.hex e DATARAM.hex) antes de realizar o script. Portanto, as regras PROGROM.hex e DATARAM.hex serão realizadas, porém, elas também possuem uma dependência do arquivo (programa).pipeline.elf. Para tanto, a ultima regra, pipeline.elf, é necessária, tendo como dependências o programa alvo compilado em arquivo objeto, o arquivo runtime, e todos os arquivos que servem como "bibliotecas" compilados também como arquivos objetos. Estas compilações são feitas através de uma regra presente no arquivo FIRMWARE/makefile.inc. Após todos, os arquivos objetos, estiverem prontos será feita a vinculação dos mesmos junto de algumas bibliotecas adicionais, ajustando também o linker script (pipeline.ld).

Voltando agora para as regras PROGROM.hex e DATARAM.hex, com o arquivo (programa).pipeline.elf pronto, serão feitas as adaptações do arquivo .elf para dois arquivos de memória formatados com 32bits de informação, em hexadecimal, por linha e escritas em ASCII. Por fim, com as dependências da regra pipeline.hex prontas é escrito no arquivo SIM/firmware.txt o programa que foi compilado.

In [9]:
!cd SIM/FIRMWARE/ && make test_rdcycle.pipeline.hex

../../FIRMWARE/makefile.inc:74: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:78: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:82: warning: ignoring prerequisites on suffix rule definition
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-gcc -I. -O2 -fno-pic -march=rv32i -mabi=ilp32 -fno-stack-protector -w -Wl,--no-relax  -c test_rdcycle.c
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   start_pipeline.S -o start_pipeline.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   putchar.S -o putchar.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   wait.S -o wait.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-gcc -I. -O2 -fno-pic -march=rv32i -mabi=ilp32 -fno-stack-protector -w -Wl,--no-relax  -c print.c
/opt/riscv_toolchain/riscv64_unknownelf

Com os arquivos DATARAM.hex e PROGROM.hex prontos é possível realizar a simulação, usando o simulador Verilator através de um testbench simples que apenas gera o clock. Para executar o simulador é preciso executar o script SIM/run_verilator.sh com a configuração desejada. O script tem 2 argumentos, o primeiro é qual implementação do processador será utilizada, e a segunda se trata do modo de execução: "v" remete à verbose e apenas simula o processador direcionando a saída para o terminal, "a" remete à all e simula todos os benchmarks disponíveis (CoreMark, Raystones e Dhrystones), direcionando a saída já filtrada para os arquivos de análise, e por fim se não houver argumento é apenas executado o programa presente nos arquivos de memória, mas também direcionando a saída para os arquivos de análise.

In [11]:
!cd SIM/ && cat run_verilator.sh

PREC="PRECOMPILED"
INFO_DIR="../INFO/BENCH"
BENCH=$(cut -d. -f 1 firmware.txt)
echo $BENCH
IMPL=$(echo $1 | cut -d. -f 1 )
IMPL_T=$IMPL
NO=$(echo "$1" | grep -Eo [0-9]+)

main() {

	(cd obj_dir; rm -f *.cpp *.o *.a VSOC)
	verilator -CFLAGS '-I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF' -DBENCH -DBOARD_FREQ=10 -DCPU_FREQ=10 -DPASSTHROUGH_PLL -Wno-fatal \
		  --top-module SOC -cc -exe bench.cpp ../../FIRMWARE/LIBFEMTORV32/femto_elf.c $1
	
	(cd obj_dir; make -f VSOC.mk)
	if [ "$2" = "v" ]
	then
		obj_dir/VSOC 
	elif [ "$2" = "a" ]
	then	
		
		cp ${PREC}/RAYSTONES/DATARAM.hex ./ && cp ${PREC}/RAYSTONES/PROGROM.hex ./
		echo "raystones.pipeline.hex" > firmware.txt
		obj_dir/VSOC > ${INFO_DIR}/temp

		branch_info
		rayst_parse
		

		cp ${PREC}/DHRYSTONES/DATARAM.hex ./ && cp ${PREC}/DHRYSTONES/PROGROM.hex ./
		echo "dhrystones.pipeline.hex" > firmware.txt
		obj_dir/VSOC > ${INFO_DIR}/temp

		branch_info
		dhry_parse

		cp ${PREC}/COREMARK/DATARAM.hex ./ && cp ${PREC}/COREMARK/PROGROM.

Execução do código test_rdcycles no processador core2.

In [13]:
!cd SIM/ && ./run_verilator.sh core2.v v

test_rdcycle
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o femto_elf.o ../../FIRMWARE/LIBFEMTORV32/femto_elf.c
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o bench.o ../bench.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno

Durante a alteração de códigos, é uma boa prática realizar a limpeza de arquivos antigos através da regra clean.

In [14]:
!cd SIM/FIRMWARE && make clean

../../FIRMWARE/makefile.inc:74: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:78: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:82: warning: ignoring prerequisites on suffix rule definition
rm -f *.o *.elf *.hex *.exe *~ *.a *.bin *.list


Compilação do código raystones.c.

In [16]:
!cd SIM/FIRMWARE && make raystones.pipeline.hex

../../FIRMWARE/makefile.inc:74: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:78: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:82: warning: ignoring prerequisites on suffix rule definition
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-gcc -I. -O2 -fno-pic -march=rv32i -mabi=ilp32 -fno-stack-protector -w -Wl,--no-relax  -c raystones.c
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   start_pipeline.S -o start_pipeline.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   putchar.S -o putchar.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   wait.S -o wait.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-gcc -I. -O2 -fno-pic -march=rv32i -mabi=ilp32 -fno-stack-protector -w -Wl,--no-relax  -c print.c
/opt/riscv_toolchain/riscv64_unknownelf/bi

Simulação do raystones através do processador core2, feita de maneira a direcionar a saída para o terminal.

In [17]:
!cd SIM/ && ./run_verilator.sh core2.v v

raystones
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o femto_elf.o ../../FIRMWARE/LIBFEMTORV32/femto_elf.c
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o bench.o ../bench.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-si

In [18]:
!cd SIM/FIRMWARE/ && make clean

../../FIRMWARE/makefile.inc:74: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:78: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:82: warning: ignoring prerequisites on suffix rule definition
rm -f *.o *.elf *.hex *.exe *~ *.a *.bin *.list


Compilação do código dhrystones.c.

In [19]:
!cd SIM/FIRMWARE/ && make dhrystones.pipeline.hex

../../FIRMWARE/makefile.inc:74: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:78: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:82: warning: ignoring prerequisites on suffix rule definition
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-gcc -I. -O2 -fno-pic -march=rv32i -mabi=ilp32 -fno-stack-protector -w -Wl,--no-relax  -c dhrystones.c
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   start_pipeline.S -o start_pipeline.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   putchar.S -o putchar.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-as -march=rv32i -mabi=ilp32   wait.S -o wait.o 
/opt/riscv_toolchain/riscv64_unknownelf/bin/riscv64-unknown-elf-gcc -I. -O2 -fno-pic -march=rv32i -mabi=ilp32 -fno-stack-protector -w -Wl,--no-relax  -c print.c
/opt/riscv_toolchain/riscv64_unknownelf/b

Simulação do código dhrystones.c, com argumento verbose.

In [20]:
!cd SIM/ && ./run_verilator.sh core2.v v

dhrystones
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o femto_elf.o ../../FIRMWARE/LIBFEMTORV32/femto_elf.c
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o bench.o ../bench.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-s

In [24]:
!cd SIM/FIRMWARE && make clean

../../FIRMWARE/makefile.inc:74: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:78: warning: ignoring prerequisites on suffix rule definition
../../FIRMWARE/makefile.inc:82: warning: ignoring prerequisites on suffix rule definition
rm -f *.o *.elf *.hex *.exe *~ *.a *.bin *.list


Para a simulação do benchmar CoreMark é mais fácil a utilização do código pré-compilado presente em SIM/PRECOMPILED/COREMARK.

In [23]:
!cd SIM/PRECOMPILED/COREMARK/ && cp DATARAM.hex PROGROM.hex ../../ && echo "coremark.pipeline.hex" > ../../firmware.txt

In [ ]:
Simulação do benchmark CoreMark, com argumento verbose.

In [25]:
!cd SIM/ && ./run_verilator.sh core2.v v

coremark
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o femto_elf.o ../../FIRMWARE/LIBFEMTORV32/femto_elf.c
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -I../../FIRMWARE/LIBFEMTORV32 -DSTANDALONE_FEMTOELF  -Os -c -o bench.o ../bench.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sig